# **Segmentez des clients d'un site e-commerce**

## partie 2/4, suite : essais de modélisation en 4 dimensions

### <br> Résumé des épisodes précédents :

> &emsp; "Vous vous apprêtez à franchir un seuil, celui de la dimension inconnue, du mystère, <br>
> de l'imagination. C'est le panneau indicateur à l'entrée de la Quatrième Dimension."<br><br>


## 1 Importation des librairies, réglages


In [ ]:
import sys
import numpy as np
import pandas as pd
import random
from datetime import datetime

import matplotlib.pyplot as plt
from matplotlib.widgets import SpanSelector
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, adjusted_rand_score
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, inconsistent
from sklearn.cluster import AgglomerativeClustering, DBSCAN
from sklearn.model_selection import GridSearchCV
from sklearn.manifold import TSNE

from IPython.display import Image
# from zipfile import ZipFile

print('Python version ' + sys.version)  # Print Python version
print('\npandas version ' + pd.__version__)
print('sns version ' + sns.__version__)

plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)
sns.set(font_scale=1)


## Fonctions


In [ ]:
def quick_look(df, miss=True):
    """
    Display a quick overview of a DataFrame, including shape, head, tail, unique values, and duplicates.

    Args:
        df (pandas.DataFrame): The input DataFrame to inspect.
        check_missing (bool, optional): Whether to check and display missing values (default is True).

    The function provides a summary of the DataFrame, including its shape, the first and last rows, the count of unique values per column, and the number of duplicates.
    If `check_missing` is set to True, it also displays missing value information.
    """
    print(f'shape : {df.shape}')

    display(df.head())
    display(df.tail())

    print('uniques :')
    display(df.nunique())

    print('Doublons ? ', df.duplicated(keep='first').sum(), '\n')

    if miss:
        display(get_missing_values(df))


def lerp(a, b, t):
    """
    Linear interpolation between two values 'a' and 'b' at a parameter 't'.
    A very useful little function, used here to position annotations in plots.
    Got it coding with Radu :)

    Given two values 'a' and 'b', and a parameter 't',
    this function calculates the linear interpolation between 'a' and 'b' at 't'.

    Parameters:
    a (float or int): The start value.
    b (float or int): The end value.
    t (float): The interpolation parameter (typically in the range [0, 1], but can be outside).

    Returns:
    float or int: The interpolated value at parameter 't'.
    """
    return a + (b - a) * t


def generate_random_pastel_colors(n):
    """
    Generates a list of n random pastel colors, represented as RGBA tuples.

    Parameters:
    n (int): The number of pastel colors to generate.

    Returns:
    list: A list of RGBA tuples representing random pastel colors.

    Example:
    >>> generate_random_pastel_colors(2)
    [(0.749, 0.827, 0.886, 1.0), (0.886, 0.749, 0.827, 1.0)]
    """
    colors = []
    for _ in range(n):
        # Generate random pastels
        red = round(random.randint(150, 250) / 255.0, 3)
        green = round(random.randint(150, 250) / 255.0, 3)
        blue = round(random.randint(150, 250) / 255.0, 3)

        # Create an RGB color tuple and add it to the list
        color = (red,green,blue, 1.0)
        colors.append(color)

    return colors

print(generate_random_pastel_colors(2))


def get_missing_values(df):
    """Generates a DataFrame containing the count and proportion of missing values for each feature.

    Args:
        df (pandas.DataFrame): The input DataFrame to analyze.

    Returns:
        pandas.DataFrame: A DataFrame with columns for the feature name, count of missing values,
        count of non-missing values, proportion of missing values, and data type for each feature.
    """
    # Count the missing values for each column
    missing = df.isna().sum()

    # Calculate the percentage of missing values
    percent_missing = df.isna().mean() * 100

    # Create a DataFrame to store the results
    missings_df = pd.DataFrame({
        'column_name': df.columns,
        'missing': missing,
        'present': df.shape[0] - missing,  # Count of non-missing values
        'percent_missing': percent_missing.round(2),  # Rounded to 2 decimal places
        'type': df.dtypes
    })

    # Sort the DataFrame by the count of missing values
    missings_df.sort_values('missing', inplace=True)

    return missings_df

# with pd.option_context('display.max_rows', 1000):
#   display(get_missing_values(df))


# ma fonction d'origine (non cleanée)
def hist_distrib(dataframe, feature, bins, r, density=True):
    """
    Affiche un histogramme, pour visualiser la distribution empirique d'une variable
    Argument : df, feature num
    """
    # calcul des tendances centrales :
    mode =  str(round(dataframe[feature].mode()[0], r))
    # mode is often zero, so Check if there are non nul values in the column
    if (dataframe[feature] != 0).any():
        mode_non_nul = str(round(dataframe.loc[dataframe[feature] != 0, feature].mode()[0], r))
    else:
        mode_non_nul = "N/A"
    mediane = str(round(dataframe[feature].median(), r))
    moyenne = str(round(dataframe[feature].mean(), r))
    # dispersion :
    var_emp = str(round(dataframe[feature].var(ddof=0), r))
    coeff_var =  str(round(dataframe[feature].std(ddof=0), r)) # = écart-type empirique / moyenne
    # forme
    skewness = str(round(dataframe[feature].skew(), 2))
    kurtosis = str(round(dataframe[feature].kurtosis(), 2))

    fig, ax = plt.subplots(figsize=(12, 5))
    dataframe[feature].hist(density=density, bins=bins, ax=ax)
    yt = plt.yticks()
    y = lerp(yt[0][0], yt[0][-1], 0.8)
    t = y/20
    xt = plt.xticks()
    x = lerp(xt[0][0], xt[0][-1], 0.7)
    plt.title(feature, pad=20, fontsize=18)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    fs =13
    plt.annotate('Mode : ' + mode, xy = (x, y), fontsize = fs, xytext = (x, y), color = 'g')
    plt.annotate('Mode + : ' + mode_non_nul, xy = (x, y-t), fontsize = fs, xytext = (x, y-t), color = 'g')
    plt.annotate('Médiane : ' + mediane, xy = (x, y-2*t), fontsize = fs, xytext = (x, y-2*t), color = 'g')
    plt.annotate('Moyenne : ' + moyenne, xy = (x, y-3*t), fontsize = fs, xytext = (x, y-3*t), color = 'g')

    plt.annotate('Var emp : ' + var_emp, xy = (x, y-5*t), fontsize = fs, xytext = (x, y-5*t), color = 'g')
    plt.annotate('Coeff var : ' + coeff_var, xy = (x, y-6*t), fontsize = fs, xytext = (x, y-6*t), color = 'g')

    plt.annotate('Skewness : ' + skewness, xy = (x, y-8*t), fontsize = fs, xytext = (x, y-8*t), color = 'g')
    plt.annotate('Kurtosis : ' + kurtosis, xy = (x, y-9*t), fontsize = fs, xytext = (x, y-9*t), color = 'g')
    plt.show()

    return float(skewness) # pour eventuel passage au log

# version cleanée
def hist_distrib(dataframe, feature, bins, decimal_places, density=True):
    """
    Visualize the empirical distribution of a numerical feature using a histogram.
    Calcul des principaux indicateurs de tendance centrale, dispersion et forme.

    Args:
        dataframe (pandas.DataFrame): The input DataFrame containing the feature.
        feature (str): The name of the numerical feature to visualize.
        bins (int): The number of bins for the histogram.
        decimal_places (int): The number of decimal places for rounding numeric values.
        density (bool, optional): Whether to display the histogram as a density plot (default is True).

    Returns:
        float: The skewness of the feature's distribution.

    The function generates a histogram of the feature, displays various statistics, and returns the skewness of the distribution.
    """
    # Calculate central tendencies and dispersion
    mode_value = round(dataframe[feature].mode()[0], decimal_places)
    mode_non_zero = "N/A"
    if (dataframe[feature] != 0).any():
        mode_non_zero = round(dataframe.loc[dataframe[feature] != 0, feature].mode()[0], decimal_places)
    median_value = round(dataframe[feature].median(), decimal_places)
    mean_value = round(dataframe[feature].mean(), decimal_places)

    # Calculate dispersion
    var_emp = round(dataframe[feature].var(ddof=0), decimal_places)
    coeff_var = round(dataframe[feature].std(ddof=0), decimal_places)

    # Calculate shape indicators
    skewness_value = round(dataframe[feature].skew(), 2)
    kurtosis_value = round(dataframe[feature].kurtosis(), 2)

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 5))
    dataframe[feature].hist(density=density, bins=bins, ax=ax)

    # Adjust placement for annotations
    yt = plt.yticks()
    y_position = lerp(yt[0][0], yt[0][-1], 0.8)
    y_increment = y_position / 20
    xt = plt.xticks()
    x_position = lerp(xt[0][0], xt[0][-1], 0.7)

    # Add annotations with horizontal and vertical alignment
    annotation_fs = 13
    color = 'g'
    ax.annotate(f'Mode: {mode_value}', xy=(x_position, y_position), fontsize=annotation_fs,
                xytext=(x_position, y_position), color=color, ha='left', va='bottom')
    ax.annotate(f'Mode +: {mode_non_zero}', xy=(x_position, y_position - y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Median: {median_value}', xy=(x_position, y_position - 2 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 2 * y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Mean: {mean_value}', xy=(x_position, y_position - 3 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 3 * y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Var Emp: {var_emp}', xy=(x_position, y_position - 5 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 5 * y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Coeff Var: {coeff_var}', xy=(x_position, y_position - 6 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 6 * y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Skewness: {skewness_value}', xy=(x_position, y_position - 8 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 8 * y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Kurtosis: {kurtosis_value}', xy=(x_position, y_position - 9 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 9 * y_increment), color=color, ha='left', va='bottom')

    # Label the x-axis and y-axis
    ax.set_xlabel(feature, fontsize=12)
    ax.set_ylabel('Frequency', fontsize=12)

    # Show the plot
    plt.title(f'Distribution of {feature}', pad=20, fontsize=18)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()

    return skewness_value


def boxplot_distrib(dataframe, feature):
    """
    Affiche un boxplot, pour visualiser les tendances centrales et la dispersion d'une variable.

    Args:
        dataframe (pandas.DataFrame): The input DataFrame containing the feature.
        feature (str): The name of the numerical feature to visualize.

    The function generates a box plot of the feature to display central tendencies (median and mean) and dispersion.
    """
    fig, ax = plt.subplots(figsize=(10, 4))

    medianprops = {'color':"blue"}
    meanprops = {'marker':'o', 'markeredgecolor':'black',
            'markerfacecolor':'firebrick'}

    dataframe.boxplot(feature, vert=False, showfliers=False, medianprops=medianprops, patch_artist=True, showmeans=True, meanprops=meanprops)

    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()


def courbe_lorenz(dataframe, feature):
    """
    Affiche une courbe de Lorenz, pour visualiser la concentration d'une variable
    Calcule l'indice de Gini
    Visualize a Lorenz curve to assess the concentration of a variable and calculate the Gini coefficient.

    Args:
        dataframe (pandas.DataFrame): The input DataFrame containing the feature.
        feature (str): The name of the numerical feature to visualize.

    The function generates a Lorenz curve to assess the concentration of the feature and calculates the Gini coefficient.
    """
    fig, ax = plt.subplots(figsize=(12, 5))
    values = dataframe.loc[dataframe[feature].notna(), feature].values
    # print(values)
    n = len(values)
    lorenz = np.cumsum(np.sort(values)) / values.sum()
    lorenz = np.append([0],lorenz) # La courbe de Lorenz commence à 0

    xaxis = np.linspace(0-1/n,1+1/n,n+1)
    #Il y a un segment de taille n pour chaque individu, plus 1 segment supplémentaire d'ordonnée 0.
    # #Le premier segment commence à 0-1/n, et le dernier termine à 1+1/n.
    plt.plot(xaxis,lorenz,drawstyle='steps-post')
    plt.plot(np.arange(2),[x for x in np.arange(2)])
    # calcul de l'indice de Gini
    AUC = (lorenz.sum() -lorenz[-1]/2 -lorenz[0]/2)/n # Surface sous la courbe de Lorenz. Le premier segment (lorenz[0]) est à moitié en dessous de 0, on le coupe donc en 2, on fait de même pour le dernier segment lorenz[-1] qui est à moitié au dessus de 1.
    S = 0.5 - AUC # surface entre la première bissectrice et le courbe de Lorenz
    gini = 2*S
    plt.annotate('gini =  ' + str(round(gini, 2)), xy = (0.04, 0.88), fontsize = 13, xytext = (0.04, 0.88), color = 'g')
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()


def graphs_analyse_uni(dataframe, feature, bins=50, r=5, density=True):
    """
    Affiche histogramme + boxplot + courbe de Lorenz

    Args:
        dataframe (pandas.DataFrame): The input DataFrame containing the feature.
        feature (str): The name of the numerical feature to analyze.
        bins (int, optional): The number of bins for the histogram (default is 50).
        decimal_places (int, optional): The number of decimal places for rounding numeric values (default is 5).
        density (bool, optional): Whether to display the histogram as a density plot (default is True).

    The function generates and displays an analysis of the given numerical feature, including an histogram, a box plot, and a Lorenz curve.
    """
    hist_distrib(dataframe, feature, bins, r)
    boxplot_distrib(dataframe, feature)
    courbe_lorenz(dataframe, feature)


def shape_head(df, nb_rows=5):
    """
    Affiche les dimensions et les premières lignes dùun dataframe
    Display the dimensions and the first rows of a DataFrame.

    Args:
        df (pandas.DataFrame): The input DataFrame to display.
        nb_rows (int, optional): The number of rows to display (default is 5, max is 60).

    The function prints the dimensions of the DataFrame and displays the first few rows.
    """
    print(df.shape)
    display(df.head(nb_rows))


def doughnut(df, feature, title, width=10, height=10):
    """
    Affiche la répartition d'une feature sous forme de diagramme circulaire
    Display the distribution of a feature as a doughnut chart.
    Les couleurs sont aléatoires.

    Args:
        df (pandas.DataFrame): The input DataFrame containing the feature.
        feature (str): The name of the feature to visualize.
        title (str): The title for the doughnut chart.
        width (int, optional): The width of the chart (default is 10).
        height (int, optional): The height of the chart (default is 10).

    The function creates a doughnut chart to visualize the distribution of the specified feature.
    If you don't like the colors, try running it again :)
    """
    colors = generate_random_pastel_colors(20)

    grouped_df = df.groupby(feature).size().to_frame("count_per_type").reset_index()
    pie = grouped_df.set_index(feature).copy()

    fig, ax = plt.subplots(figsize=(width, height))

    patches, texts, autotexts = plt.pie(x=pie['count_per_type'], autopct='%1.1f%%',
        startangle=-30, labels=pie.index, textprops={'fontsize':11, 'color':'#000'},
        labeldistance=1.25, pctdistance=0.85, colors=colors)

    plt.title(
    label=title,
    fontdict={"fontsize":17},
    pad=20
    )

    for text in texts:
        # text.set_fontweight('bold')
        text.set_horizontalalignment('center')

    # Customize percent labels
    for autotext in autotexts:
        autotext.set_horizontalalignment('center')
        autotext.set_fontstyle('italic')
        autotext.set_fontsize('10')

    #draw circle
    centre_circle = plt.Circle((0,0),0.7,fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)

    plt.show()


def get_non_null_values(df):
    """
    Génère un dataframe contenant le nombre et la proportion de non-null (non-zero) valeurs pour chaque feature
    Generate a DataFrame containing the count and proportion of non-null (non-zero) values for each feature.

    Args:
        df (pandas.DataFrame): The input DataFrame to analyze.

    The function calculates and returns a DataFrame with the count and percentage of non-null (non-zero) values for each feature.
    """
    non_null_counts = df.ne(0).sum()
    percent_non_null = (non_null_counts / df.shape[0]) * 100
    non_null_values_df = pd.DataFrame({'column_name': df.columns,
                                       'non_null_count': non_null_counts,
                                       'percent_non_null': percent_non_null.round(2),
                                       'type': df.dtypes})
    non_null_values_df.sort_values('non_null_count', inplace=True)
    return non_null_values_df


def get_colors(n=7):
    """
    Generate a list of random colors from multiple colormaps.

    Args:
        n (int, optional): The number of colors to sample from each colormap (default is 7).

    Returns:
        list: A list of random colors sampled from different colormaps.
    """
    num_colors_per_colormap = n
    colormaps = [plt.cm.Pastel2, plt.cm.Set1, plt.cm.Paired]
    all_colors = []

    for colormap in colormaps:
        colors = colormap(np.linspace(0, 1, num_colors_per_colormap))
        all_colors.extend(colors)

    np.random.shuffle(all_colors)

    return all_colors


## Import


In [ ]:
rfms_complet = pd.read_csv('data/rfms.csv', sep=',')

rfms_complet['datetime'] = pd.to_datetime(rfms_complet['datetime'])

quick_look(rfms_complet, True)


In [ ]:
rfms = rfms_complet.loc[rfms_complet['satisfaction'].notna(), :].copy()

quick_look(rfms, True)


## Visualisation


### toute la data


In [ ]:
# Create a 3D scatter plot
fig = px.scatter_3d(rfms, x='recent_timestamp', y='nb_commandes', z='payment_total')

# Customize the plot
fig.update_traces(marker=dict(size=5))
fig.update_layout(title='Our data in 3D')

fig.update_layout(
    width=800,  # Specify the width in pixels
    height=600  # Specify the height in pixels
)

fig.show()


### Nouvelle feature : review score


In [ ]:
rfms['round_sat'] = np.round(rfms['satisfaction']) # for simple visualisation
hist_distrib(rfms, 'round_sat', 5, 2)

# Colorons maintenant en fonction du review_score
custom_color_set_5 = ['red', 'purple', 'blue', 'lightgreen', 'yellow']

fig, ax = plt.subplots(figsize=(12, 5))
N, bins, patches = ax.hist(rfms['round_sat'], density=False, bins=5)
plt.title('review_scores moyens', pad=20, fontsize=18)

for i, color in enumerate(custom_color_set_5):
    patches[i].set_facecolor(color)

plt.show()


### tests visualisation


In [ ]:
# Problème couleurs

def plot_data_3D_hue(df=rfms, legend=True, color_set=custom_color_set_5, opacity=1.0):
    # np.sort(df['round_sat'].unique())
    labels_str = df['round_sat'].astype(str)
    colors = color_set[:len(set(df['round_sat']))]
    # Create a 3D scatter plot with custom legend
    fig = px.scatter_3d(df, x='recent_timestamp', y='nb_commandes', z='payment_total', color=labels_str,
                        color_discrete_sequence=colors, opacity=opacity)

    fig.update_traces(marker=dict(size=5), showlegend=legend)
    fig.update_layout(title='Our data')

    fig.update_layout(
        width=800,  # Specify the width in pixels
        height=600  # Specify the height in pixels
    )
    fig.show()

plot_data_3D_hue()


In [ ]:
# OK

def plot_data_3D_hue(df=rfms):
    subset = []
    fig = go.Figure()

    for i in range(1, 6):
        subset.append(df.loc[df['round_sat'] == i, :].copy())
        print(f'Subset review_score moyen == {i} : {subset[i-1].shape[0]} clients')

        scatter = go.Scatter3d(x=subset[i-1]['recent_timestamp'],
                            y=subset[i-1]['nb_commandes'],
                            z=subset[i-1]['payment_total'],
                            mode='markers',
                            marker=dict(size=5, color=custom_color_set_5[i-1]),
                            name=f'score={i}')  # Set the legend label here
        fig.add_trace(scatter)

    fig.update_layout(title=f'Our data in 3D+ Subset {i}')
    fig.update_layout(width=600, height=400)
    fig.update_scenes(xaxis_title_text='recent_timestamp', # Set the labels
                yaxis_title_text='nb_commandes',
                zaxis_title_text='payment_total')

    fig.show()

plot_data_3D_hue()


In [ ]:
def plot_data_4D(df=rfms):
    subset = []

    for i in range(1, 6):
        subset.append(df.loc[df['round_sat'] == i, :].copy())
        print(f'Subset review_score moyen == {i} : {subset[i-1].shape[0]} clients')

        fig = go.Figure()

        scatter = go.Scatter3d(x=subset[i-1]['recent_timestamp'],
                            y=subset[i-1]['nb_commandes'],
                            z=subset[i-1]['payment_total'],
                            mode='markers',
                            marker=dict(size=5, color=custom_color_set_5[i-1]),
                            name=f'score={i}')  # Set the legend label here

        fig.add_trace(scatter)
        fig.update_layout(title=f'Our data in 3D+ Subset {i}')
        fig.update_layout(width=500, height=400)
        fig.update_scenes(xaxis_title_text='recent_timestamp', # Set the labels
                    yaxis_title_text='nb_commandes',
                    zaxis_title_text='payment_total')

        fig.show()

plot_data_4D(df=rfms)

# Pas forcément évident parce qu'on a bcp de points, mais on voit ici comment la data est étirée
# le long d'un nouvel axe, à chaque fois qu'on prend en compte une nouvelle feature dans notre analyse

# En 5 dimensions, on peut encore visualiser, en affichant un plan de projections 3D
# Au-delà ça va devenir + technique avec les limitations d'un notebook.
# Note de futur : c mm pire que ce que j'imaginais

# Ici on dirait que les clients les + satisfaits (jaune) ont fait relativement peu de commandes,
# par rapport aux aux clients moins satisfaits.

# C'est à la fois frappant, car ce groupe représente + de la moitié du dataset total,
# et inquiétant pour Olis : il semble que + les clients commandent, + le review_score baisse
# (à vérifier. Si confirmation, cela demande à être étudié)


### Evolution du review score en fonction de nb de commandes


In [ ]:
rfms['count_clients'] = 1

df_satisfaction = rfms.groupby('nb_commandes').agg({
    'satisfaction': 'mean',
    'count_clients': 'count'
}).reset_index()

# feature passée au log
df_satisfaction['vrai_nb_commandes'] = np.exp(df_satisfaction['nb_commandes'])

max_nb_commandes = df_satisfaction['vrai_nb_commandes'].max()
print(max_nb_commandes)

fig, ax = plt.subplots(figsize=(12, 5))  # Adjust the figsize as needed

df_satisfaction.plot(kind='line', x='vrai_nb_commandes', y='satisfaction', fontsize=12, color='b', ax=ax)
# df_satisfaction.plot(kind='line', x='vrai_nb_commandes', y='count_clients', fontsize=12, color='r', ax=ax)
plt.title('Satisfaction / nb_commandes', fontsize=18, pad=20)
plt.legend(loc='upper left', fontsize=12)
plt.xticks(fontsize=7)
plt.show()


### Risque d'attrition


In [ ]:
df_satisfaction['count_clients'] = df_satisfaction['count_clients'] / 10000       # (visu)

fig, ax = plt.subplots(figsize=(12, 5))  # Adjust the figsize as needed

df_satisfaction.plot(kind='line', x='vrai_nb_commandes', y='satisfaction', fontsize=12, color='b', ax=ax)
df_satisfaction.plot(kind='line', x='vrai_nb_commandes', y='count_clients', fontsize=12, color='r', ax=ax)
plt.title('Satisfaction / nb_commandes', fontsize=18, pad=20)
plt.legend(loc='upper left', fontsize=12)
plt.xticks(fontsize=7)
plt.show()

# Ce n'était pas qu'une impression, la satisfaction myenne chute après 8-9 commandes,
# puis remonte régulièrement. Sans s'avancer sur des hypothèses qd aux causes,
# (par exemple, il pourrait simplement s'agir de la proba qu'une livraison finisse par mal se passer)
# je trouve que c'est un point intéressant à mentionner à Olis :
# on a une zone rouge après 7 commandes où le risque d'attrition (churning) est élevé,
# il faut donc mettre en place des mesures préventives.
# Ceci dit cela ne concerne que très peu de clients, une petite fraction des "3%" de clients "réguliers"


### Forme des samples


In [ ]:
# Notre subset au 1/10eme capture-t-il tjs la forme générale... en 4 dimensions ?

plot_data_4D(df=rfms[::10])

# Ca a l'air pas trop mal, sauf pour les rouges qui perdent les valeurs moyennes sur l'axe nb_commandes


In [ ]:
# Essayons d'ajouter de l'aléatoire dans notre méthode de sélection

features_RFMS = ['recent_timestamp', 'nb_commandes', 'payment_total', 'satisfaction']

# Shuffle the DataFrame randomly
shuffled_df = rfms[features_RFMS + ['round_sat']].sample(frac=1, random_state=i)  # frac=1 shuffles all rows

# Sample a portion of the shuffled DataFrame
sample_size = 10000
sampled_df = shuffled_df.head(sample_size)

plot_data_4D(df=sampled_df)


In [ ]:
# On a tjs des trous, en 4D le nombre d'individus n'est plus suffisant pour couvrir l'espace.

sample_size = 20000
sampled_df = shuffled_df.head(sample_size)

plot_data_4D(df=sampled_df)


In [ ]:
# Même au 1/5ème, la topologie est déformée. Des vides apparaissent.


### Scaling


In [ ]:
scaler = StandardScaler()

rfms_scaled = rfms.copy()

rfms_scaled[features_RFMS + ['round_sat']] = scaler.fit_transform(rfms[features_RFMS + ['round_sat']])

hist_distrib(rfms_scaled, 'satisfaction', 5, 2)
hist_distrib(rfms_scaled, 'round_sat', 5, 2)
# OK, same distribs

# C quoi la fonction de normalisation qd une feature a un skew négatif (asymétrie left tail) ?


## 2 kmeans


### nombre optimal de clusters using different scores


In [ ]:
# Plot the score values for different numbers of clusters
def plot_score(score_list, score_name, max=15):
    """
    Plot the score values for different numbers of clusters.

    Args:
        score_list (list): A list of score values for different numbers of clusters.
        score_name (str): The name of the score being plotted (e.g., 'Silhouette Score').
        max (int, optional): The maximum number of clusters to plot (default is 15).

    This function generates a line plot to visualize how a specific clustering score varies with different numbers of clusters (k).
    It helps in identifying the optimal number of clusters using the given clustering score.

    Parameters:
    - score_list: A list of score values for different values of k.
    - score_name: The name of the score to display in the plot (e.g., 'Silhouette Score', 'Inertia', etc.).
    - max: The maximum number of clusters to plot (default is 15).
    """
    plt.figure(figsize=(8, 6))
    plt.plot(range(2, max), score_list, marker='o', linestyle='-', color='b')
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel(score_name)
    plt.title('Optimal k, using ' + score_name + ' score')
    plt.grid(True)
    plt.show()

# Calculate Tightness
def compute_tightness(data, cluster_labels, cluster_centers):
    """
    Calculate the tightness score for a clustering.

    Args:
        data (numpy.ndarray): The data points.
        cluster_labels (numpy.ndarray): Labels indicating the cluster assignment for each data point.
        cluster_centers (numpy.ndarray): The centers of the clusters.

    Returns:
        float: The tightness score, a measure of how close data points are to their cluster centers.

    The tightness score quantifies the compactness of clusters in a clustering. It is computed as the average distance of data points within each cluster to their respective cluster center. A lower tightness score indicates that data points are closer to their cluster centers, indicating tighter clusters.

    Parameters:
    - data: The data points in the dataset.
    - cluster_labels: An array of cluster labels for each data point.
    - cluster_centers: The centers of the clusters.
    """
    tightness_score = 0
    for i in range(len(cluster_centers)):
        cluster_points = data[cluster_labels == i]
        center = cluster_centers[i]
        distances = np.linalg.norm(cluster_points - center, axis=1)
        tightness_score += np.mean(distances)
    tightness_score /= len(cluster_centers)
    return tightness_score

def find_best_nb_clusters_for_kmeans(df=rfms_scaled, features=features_RFMS, scores=[], max=15):
    """
    Find the best number of clusters for K-Means clustering based on various scoring methods.

    Args:
        df (pd.DataFrame): The dataset for clustering.
        features (list): List of feature columns for clustering.
        scores (list): List of scoring methods to use ('inertia', 'tightness', 'db', 'silhouette').
        max (int): The maximum number of clusters to consider (default is 15).

    This function performs K-Means clustering with different numbers of clusters (k) and evaluates the clustering results using various scoring methods, including inertia, tightness, Davies-Bouldin index, and silhouette score. It then plots the scores to help determine the optimal number of clusters based on the chosen scoring methods.

    Parameters:
    - df: The dataset for clustering.
    - features: List of feature columns for clustering.
    - scores: List of scoring methods to use ('inertia', 'tightness', 'db', 'silhouette').
    - max: The maximum number of clusters to consider (default is 15).
    """
    # Create empty lists to store scores for different k values
    inertia_scores = []
    tightness_scores = []
    davies_bouldin_scores = []
    silhouette_scores = []

    # Test different numbers of clusters from 2 to 15
    for k in range(2, max):
        kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
        kmeans.fit(df[features])

        if 'inertia' in scores:
            inertia_scores.append(kmeans.inertia_)
        if 'tightness' in scores:
            tightness = compute_tightness(df[features], kmeans.labels_, kmeans.cluster_centers_)
            tightness_scores.append(tightness)
        if 'db' in scores:
            davies_bouldin = davies_bouldin_score(df[features], kmeans.labels_)
            davies_bouldin_scores.append(davies_bouldin)
        if 'silhouette' in scores:
            silhouette = silhouette_score(df[features], kmeans.labels_)
            silhouette_scores.append(silhouette)

    if 'inertia' in scores:
        plot_score(inertia_scores, 'inertia', max=max)
    if 'tightness' in scores:
        plot_score(tightness_scores, 'tightness', max=max)
    if 'db' in scores:
        plot_score(davies_bouldin_scores, 'Davies-Bouldin', max=max)
    if 'silhouette' in scores:
        plot_score(silhouette_scores, 'silhouette', max=max)


find_best_nb_clusters_for_kmeans(scores=['inertia'], max=15)

# best nb de cluster : 5 (elbow method)
# tps : 6s


In [ ]:
find_best_nb_clusters_for_kmeans(scores=['tightness'], max=15)

# best nb de cluster : 5-6 (coude), 11 (min)
# tps : 6-7 s


In [ ]:
find_best_nb_clusters_for_kmeans(df=rfms_scaled, scores=['db'], max=15)

# best nb de cluster : 5 (min)
# tps : 6s


### Silhouette score


In [ ]:
# Sur dataset complet, compter 68 minutes (thermostat 6)
# find_best_nb_clusters_for_kmeans(df=rfms_scaled, scores=['silhouette'], max=15)

# Sur le subset 1/10, 20-25 secondes
# La courbe est assez différente
# find_best_nb_clusters_for_kmeans(df=rfms_scaled[::10], scores=['silhouette'], max=15)

# best nb de cluster : 5 (max, elbow)
display(Image(filename='img/silhouette_4D.png'))


### Yellowbrick


In [ ]:
# Instantiate the clustering model and visualizer
model = KMeans(n_init='auto')
visualizer = KElbowVisualizer(model, k=(2,12))

visualizer.fit(rfms_scaled[features_RFMS])    # Fit the data to the visualizer
visualizer.poof()    # Draw/show/poof the data

# bcp, bcp + rapide que notre méthode (silhouette_score, from sklearn.metrics)
# Comment ils font ?

# best nb clusters = 5


In [ ]:
# Instantiate the clustering model and visualizer
model = KMeans(n_init='auto')
visualizer = KElbowVisualizer(model, k=(2,12), metric='calinski_harabasz', timings=False)

visualizer.fit(rfms_scaled[features_RFMS])    # Fit the data to the visualizer
visualizer.poof()    # Draw/show/poof the data

# 5


In [ ]:
# Beaucoup de scores concordants pour best number of clusters = 5 ici

model = KMeans(5, n_init='auto')
visualizer = SilhouetteVisualizer(model)

# environ 6 min sur dataset complet
# visualizer.fit(rfms_scaled[features_RFMS])    # Fit the data to the visualizer
# visualizer.poof()    # Draw/show/poof the data

# tjs pas terrible (en termes de séparation / régularité des clusters)
# (1 cluster a bcp moins d'étendue, d'inertie) (= nos 3% ?)
# comparer au resultat avec 4 clusters (oubli, en-dessous) : bcp + équilibré using best k :)

display(Image(filename='img/poof_rfms5.png'))
display(Image(filename='img/poof_rfms4.png'))


### Stabilité


In [ ]:
# Etude de la stabilité des clusters sur une sous-partition aléatoire
# le nombre de clusters proposés reste (étonnament ?) stable

def not_sure_why_im_doin_this_one():
    n_iter = 10

    for i in range(n_iter):
        # Shuffle the DataFrame randomly
        shuffled_df = rfms_scaled[features_RFMS].sample(frac=1, random_state=i)  # frac=1 shuffles all rows

        # Sample a portion of the shuffled DataFrame
        sample_size = 5000  # Adjust this to your desired sample size
        sampled_df = shuffled_df.head(sample_size)

        inertia = []

        for k in range(2, 11):
            kmeans = KMeans(n_clusters=k, n_init=10, random_state=i)
            kmeans.fit(shuffled_df[features_RFMS])

            inertia.append(kmeans.inertia_)

        print(inertia)

        # Plot the inertia values for different numbers of clusters
        plt.figure(figsize=(8, 6))
        plt.plot(range(2, 11), inertia, marker='o', linestyle='-', color='b')
        plt.xlabel('Number of Clusters (k)')
        plt.ylabel('Inertia')
        plt.title('Elbow Method for Optimal k')
        plt.grid(True)

# not_sure_why_im_doin_this_one()

# Tjs très stable !
# Vérifier mieux que les clusters sont proches, similaires : -> utiliser l'ARI


### ARI


In [ ]:
# Nous allons avoir besoin d'une fonction qui retourne les labels assignés par le k-means
# (pour visualisation et ARI).

# Assuming 5 is the optimal number of clusters
def assign_labels_using_kmeans(alea, df=rfms_scaled, features=features_RFMS, best_k=5, silhouette=False):
    """
    Assign cluster labels to data points using K-Means clustering.

    Args:
        alea (int): Random seed for reproducibility.
        df (pd.DataFrame): The dataset for clustering.
        features (list): List of feature columns for clustering.
        best_k (int): The number of clusters for K-Means (default is 5).
        silhouette (bool): Whether to print the silhouette score for the K-Means clustering (default is False).

    This function performs K-Means clustering on the given data with a specified number of clusters (best_k) and assigns cluster labels to data points. Optionally, it can print the silhouette score as a measure of clustering quality.

    Returns:
    - labels (array): Cluster labels assigned to data points.
    """
    kmeans = KMeans(n_clusters=best_k, n_init='auto', random_state=alea)
    kmeans.fit(df[features])

    if silhouette == True:
        print('silhouette_score for kmeans = ', silhouette_score(df[features], kmeans.labels_))

    return kmeans.labels_

# L'ARI est souvent utilisé pour comparer le clustering obtenu par un modèle à une partition connue,
# pour tester la qualité du modèle. Ici c'est un peu différent,
# Nous n'avons pas de partition pré-établie. Nous allons seulement tester la stabilité,
# pas la qualité, en comparant les labels obtenus successivement à ceux de l'essai précédent.

labels_kmeans = []

for i in range(0, 15):
    labels = assign_labels_using_kmeans(alea=i)
    labels_kmeans.append(labels)
    if i > 0:
        # Calculate ARI
        ari = adjusted_rand_score(labels_kmeans[i], labels_kmeans[i-1])
        print(f'ARI = {ari}', '\n')

display(labels_kmeans)

# Tous les k-means se ressemblent.


### Visualisation, premier clustering RFMS


In [ ]:
# Ca a l'air super stable.
# Visualisons !

custom_color_set_5 = ['red', 'pink', 'lightgreen', 'yellow', 'blue']

def plot_kmeans_3D(df=rfms_scaled, legend=False):
    fig = px.scatter_3d(df, x='recent_timestamp', y='nb_commandes', z='payment_total', color=labels_kmeans[0],
                        color_discrete_sequence=custom_color_set_5, opacity=1)

    fig.update_traces(marker=dict(size=5), showlegend=False)
    fig.update_layout(title='K-Means Clustering in 3D + hue')

    fig.update_layout(
        width=800,  # Specify the width in pixels
        height=600  # Specify the height in pixels
    )
    fig.show()

plot_kmeans_3D()

# la légende est displayed qd mm ?
# Couleurs pas ds le bon ordre

# Je distingue 4 clusters, mais je ne vois pas bien le 5ème (en jaune)
# Ce cluster est peut-être davantage lié à notre 4ème dimension, qui n'est pas représentée ici.

# Les 4 clusters visibles rappelent fortement notre segmentation RFM


In [ ]:
# Changeons d'Axes3D

cluster_colors = custom_color_set_5[:len(set(labels_kmeans[0]))]
rfms_scaled['labels'] = labels_kmeans[0].astype(int).astype(str)

fig = px.scatter_3d(rfms_scaled, x='recent_timestamp', y='satisfaction', z='payment_total', color='labels',
                            color_discrete_sequence=cluster_colors, opacity=0.9)

fig.update_traces(marker=dict(size=5), showlegend=True)
fig.update_layout(title='K-Means Clustering in 4D, low resolution')

fig.update_layout(
    width=800,  # Specify the width in pixels
    height=600  # Specify the height in pixels
)
fig.show()


In [ ]:
# La légende ne sert pas à gd chose ici, c + clair sans.
# Ou en mode discret (catégorique)
# couleurs OK, legend à ordonner

def plot_kmeans_3D(df=rfms_scaled, legend=True):
    # Convert cluster labels to integer and then to string format
    cluster_labels_str = labels_kmeans[0].astype(int).astype(str)
    # Create a list of colors based on labels
    cluster_colors = custom_color_set_5[:len(set(labels_kmeans[0]))]

    # Create a 3D scatter plot with custom legend
    fig = px.scatter_3d(df, x='recent_timestamp', y='nb_commandes', z='payment_total', color=cluster_labels_str,
                        color_discrete_sequence=cluster_colors, opacity=0.9)

    fig.update_traces(marker=dict(size=5), showlegend=legend)
    fig.update_layout(title='K-Means Clustering in 3D')

    fig.update_layout(
        width=800,  # Specify the width in pixels
        height=600  # Specify the height in pixels
    )
    fig.show()

plot_kmeans_3D(legend=True)

# On peut tjs essayer de comprendre une partie de notre clustering directement :
# le cluster bleu (2) semble tjs regrouper les clients les plus réguliers.

# Cependant les autres clusters sont plus difficiles à distinguer, donc
# plus difficiles à interpréter.


In [ ]:
# legende tjs désordre

def plot_clusters_4D(df=rfms_scaled, legend=True):
    subset = []
    # Create a list of colors based on labels
    cluster_colors = custom_color_set_5[:len(set(labels_kmeans[0]))]

    unique_values = np.sort(df['round_sat'].unique())
    print(unique_values)

    for i, score in enumerate(unique_values):
        subset.append(df.loc[df['round_sat'] == score, :].copy())

        print(f'Subset review_score moyen == {score} : {subset[i-1].shape[0]} clients')

        # Create a 3D scatter plot with custom legend
        fig = px.scatter_3d(subset[i-1], x='recent_timestamp', y='nb_commandes', z='payment_total', color='labels',
                            color_discrete_sequence=cluster_colors, opacity=0.9)

        fig.update_traces(marker=dict(size=5), showlegend=legend)
        fig.update_layout(title='K-Means Clustering in 4D, low resolution')

        fig.update_layout(
            width=600,  # Specify the width in pixels
            height=400  # Specify the height in pixels
        )
        fig.show()

plot_clusters_4D()


In [ ]:
# nope, c pire...

rfms_scaled['color'] = ""

for i, label in enumerate(rfms_scaled['labels'].unique()):
    rfms_scaled.loc[rfms_scaled['labels'] == label, 'color'] = custom_color_set_5[i]

def plot_clusters_4D(df=rfms_scaled, legend=True):
    subset = []
    # Create a list of colors based on labels
    cluster_colors = custom_color_set_5[:len(set(labels_kmeans[0]))]

    unique_values = np.sort(df['round_sat'].unique())
    print(unique_values)

    for i, score in enumerate(unique_values):
        subset.append(df.loc[df['round_sat'] == score, :].copy())
        print(f'Subset review_score moyen == {score} : {subset[i-1].shape[0]} clients')

        # Create a 3D scatter plot with custom legend
        fig = px.scatter_3d(subset[i-1], x='recent_timestamp', y='nb_commandes', z='payment_total', color='color')

        fig.update_traces(marker=dict(size=5), showlegend=legend)
        fig.update_layout(title='K-Means Clustering in 4D, low resolution')

        fig.update_layout(
            width=600,  # Specify the width in pixels
            height=400  # Specify the height in pixels
        )
        fig.show()

plot_clusters_4D()


In [ ]:
# tjs pas... compliqué les couleurs avec plotly

def plot_clusters_4D(df=rfms_scaled, legend=True):
    subset = []
    # Create a list of colors based on labels
    cluster_colors = custom_color_set_5[:len(set(labels_kmeans[0]))]

    unique_values = np.sort(df['round_sat'].unique())
    print(unique_values)

    for i, score in enumerate(unique_values):
        subset.append(df.loc[df['round_sat'] == score, :].copy())
        subset[i-1].sort_values(by='labels', inplace=True)

        print(f'Subset review_score moyen == {score} : {subset[i-1].shape[0]} clients')

        # Create a 3D scatter plot with custom legend
        fig = px.scatter_3d(subset[i-1], x='recent_timestamp', y='nb_commandes', z='payment_total', color='labels',
                            color_discrete_sequence=cluster_colors, opacity=0.9)

        fig.update_traces(marker=dict(size=5), showlegend=legend)
        fig.update_layout(title='K-Means Clustering in 4D, low resolution')

        fig.update_layout(
            width=600,  # Specify the width in pixels
            height=400  # Specify the height in pixels
        )
        fig.show()

plot_clusters_4D()


### Résultat


In [ ]:
# En trichant ?
# Enfin des couleurs cohérentes !
# Mauve et bleu, et jaune et pourpre.
# En fait j'aurais pu conserver la boucle et looper sur une liste de liste de couleurs

def plot_clusters_4D(df=rfms_scaled, legend=True):
    subset = []
    # Create a list of colors based on labels
    cluster_colors = custom_color_set_5[:len(set(labels_kmeans[0]))]

    unique_values = np.sort(df['round_sat'].unique())
    print(unique_values)


    # 0
    subset.append(df.loc[df['round_sat'] == unique_values[0], :].copy())
    subset[0].sort_values(by='labels', inplace=True)

    print(f'Subset review_score moyen == {unique_values[0]} : {subset[0].shape[0]} clients')

    fig = px.scatter_3d(subset[0], x='recent_timestamp', y='nb_commandes', z='payment_total', color='labels',
                        color_discrete_sequence=['yellow', 'lightgreen', 'red'], opacity=0.9)

    fig.update_traces(marker=dict(size=5), showlegend=legend)
    fig.update_layout(title='K-Means Clustering in 4D, low resolution')

    fig.update_layout(
        width=600,  # Specify the width in pixels
        height=400  # Specify the height in pixels
    )
    fig.show()

    # 1
    subset.append(df.loc[df['round_sat'] == unique_values[1], :].copy())
    subset[1].sort_values(by='labels', inplace=True)

    print(f'Subset review_score moyen == {unique_values[1]} : {subset[1].shape[0]} clients')

    fig = px.scatter_3d(subset[1], x='recent_timestamp', y='nb_commandes', z='payment_total', color='labels',
                        color_discrete_sequence=['orange', 'yellow', 'lightgreen', 'silver', 'red'], opacity=0.9)

    fig.update_traces(marker=dict(size=5), showlegend=legend)
    fig.update_layout(title='K-Means Clustering in 4D, low resolution')

    fig.update_layout(
        width=600,  # Specify the width in pixels
        height=400  # Specify the height in pixels
    )
    fig.show()

    # 2
    subset.append(df.loc[df['round_sat'] == unique_values[2], :].copy())
    subset[2].sort_values(by='labels', inplace=True)

    print(f'Subset review_score moyen == {unique_values[2]} : {subset[2].shape[0]} clients')

    # Create a 3D scatter plot with custom legend
    fig = px.scatter_3d(subset[2], x='recent_timestamp', y='nb_commandes', z='payment_total', color='labels',
                        color_discrete_sequence=['orange', 'yellow', 'lightgreen', 'silver', 'red'], opacity=0.9)

    fig.update_traces(marker=dict(size=5), showlegend=legend)
    fig.update_layout(title='K-Means Clustering in 4D, low resolution')

    fig.update_layout(
        width=600,  # Specify the width in pixels
        height=400  # Specify the height in pixels
    )
    fig.show()

    # 3
    subset.append(df.loc[df['round_sat'] == unique_values[3], :].copy())
    subset[3].sort_values(by='labels', inplace=True)

    print(f'Subset review_score moyen == {unique_values[3]} : {subset[3].shape[0]} clients')

    # Create a 3D scatter plot with custom legend
    fig = px.scatter_3d(subset[3], x='recent_timestamp', y='nb_commandes', z='payment_total', color='labels',
                        color_discrete_sequence=['orange', 'yellow', 'lightgreen', 'silver'], opacity=0.9)

    fig.update_traces(marker=dict(size=5), showlegend=legend)
    fig.update_layout(title='K-Means Clustering in 4D, low resolution')

    fig.update_layout(
        width=600,  # Specify the width in pixels
        height=400  # Specify the height in pixels
    )
    fig.show()

    # 4
    subset.append(df.loc[df['round_sat'] == unique_values[4], :].copy())
    subset[4].sort_values(by='labels', inplace=True)

    print(f'Subset review_score moyen == {unique_values[4]} : {subset[4].shape[0]} clients')

    # Create a 3D scatter plot with custom legend
    fig = px.scatter_3d(subset[4], x='recent_timestamp', y='nb_commandes', z='payment_total', color='labels',
                        color_discrete_sequence=['orange', 'yellow', 'lightgreen', 'silver'], opacity=0.9)


    fig.update_traces(marker=dict(size=5), showlegend=legend)
    fig.update_layout(title='K-Means Clustering in 4D, low resolution')

    fig.update_layout(
        width=600,  # Specify the width in pixels
        height=400  # Specify the height in pixels
    )
    fig.show()

plot_clusters_4D()


## Conclusion : DESCRIPTION ACTIONNABLE du clustering obtenu


In [ ]:
# On voit enfin tte notre data en 4D !
# Cependant, on est très limités au niveau de la "résolution"...
# J'ai essayé d'itérer sur ttes les valeurs possibles du review_score moyen, 34 valeurs en tout.
# print(rfms_scaled['satisfaction'].nunique())

# Mais la cellule fait crasher le notebook au-delà de qq plots.
# Du coup je continue à utiliser la valeur bucketisée en 5 tranches.

# Autre conséquence : on ne pourra mm pas vidualiser la 5eme dim avec un notebook,
# c'est vraiment dommage. -> chercher solutions

# On voit tt de mm qqch apparaitre ici (malgré les couleurs aui sont à nouveau mélangées...)
# ! Utiliser la legende, les couleurs changent à chaque sous-plot !
# ! CORRIGER CES FICHUES COULEURS ! ELLES COMPLIQUENT LA LECTURE AU LIEU D'AIDER

# Lecture :

# Les clusters 0 et 3 regroupent uniquement des clients "satisfaits" (review_score moyen ou +),
# c'est pourquoi ils n'apparaissent pas sur les 2 premiers plots (review_scores faibles)

# Le cluster 3 représente donc les "pire clients" (1 seule commande, au montant relativement peu élevé,
# il y a longtemps), parmi les clients satisfaits.
# = Je les appelerais le groupe des "y a pas de raison qu'ils recommandent pas, mais bon".
# Pour Olis c'est un groupe assez prometteur de bons clients potentiels, puisqu'ils ont déjà utilisé
# la plateforme et se sont déclaré satisfaits, mais sans doute à "réactiver" car
# la plupart n'ont pas recommandé depuis + d'un an.

# Le cluster 0 regroupe des clients qui eux aussi sont satisfaits, n'ont fait qu'une commande,
# au montant peu élevé, mais plus récente.
# = un groupe très prometteur, en raison surtout de la commande récente + score satisfait

# Le cluster 1 regroupe les clients qui ont dépensé le plus, ont fait 1 seule commande,
# et sont plutôt satisfaits.
# Il semble relativement indépendant de la récence et de la satisfaction
# (réparti sur toute la longueur de l'axe, de manière assez homogène)
# = Très fort potentiel, clients à cibler car potentiellement dépensiers !

# Le cluster 2 regroupe clairement les clients réguliers (2 commandes et +)

# Le 4 regroupe des clients très mécontents ou au mieux moyennement satisfaits,
# n'ayant fait qu'une seule commande, indépendamment de la récence et du montant.
# = sans doute le groupe le moins prometteur.
# Cibler ce groupe pour comprendre les causes possibles d'insatisfaction,
# et tenter de corriger ds une certaine mesure.


## 3 dendro


### sampling


In [ ]:
# Work on a very small sample (10-15s) (très déformé)
# dendro_df = rfms_scaled[features_RFMS][::50].copy()

# Work on a small sample (1 min 30s)
dendro_df = rfms_scaled[features_RFMS][::10].copy()

# Use complete data : impossible without modifying notebook params
# (dataset way too big for an algo with this order of complexity)

display(Image(filename='img/dendro_rfms.png'))

# Perform hierarchical clustering
linkage_matrix = linkage(dendro_df, method='ward')  # You can choose different linkage methods

# Create a dendrogram
dendrogram(linkage_matrix, labels=dendro_df.index.values)

# Display the dendrogram
plt.title("Dendrogram")
plt.xlabel("Sample Index")
plt.ylabel("Distance")

plt.axhline(y=82, color='orange', linestyle='--', label='Threshold')
# best number of clusters = 4 with this first "method"
# (More like a quick empirical "rule of thumb")

plt.show()

# Bcp + long que k-means, compliqué pour travailler sur le dataset complet...
# Propose 4-5 clusters


### nb optimal de cluster, using inconsistency


In [ ]:
# Determine the optimal number of clusters using inconsistency
depth = 5  # You can adjust this depth parameter
inconsistencies = inconsistent(linkage_matrix, depth)
optimal_cluster_count = inconsistencies.argmax() + 1

# Plot the inconsistency values
plt.figure()
plt.plot(range(2, len(inconsistencies) + 2), inconsistencies)
plt.title("Inconsistency vs. Number of Clusters")
plt.xlabel("Number of Clusters")
plt.ylabel("Inconsistency")
plt.axvline(x=optimal_cluster_count, color='red', linestyle='--', label='Optimal Cluster Count')
plt.legend()
plt.show()


In [ ]:
# 38 000 clusters, c peut-être mathématiquement optimal, mais...
# Bon courage pour l'interprétation !

# Plot the inconsistency values for up to 15 clusters
max_clusters_to_plot = 15
optimal_cluster_count = inconsistencies[:max_clusters_to_plot - 1].argmax() + 2 # index starts at 2

plt.figure()

plt.plot(range(2, max_clusters_to_plot + 2), inconsistencies[:max_clusters_to_plot])

plt.title("Inconsistency vs. Number of Clusters")
plt.xlabel("Number of Clusters")
plt.ylabel("Inconsistency")
plt.axvline(x=optimal_cluster_count, color='red', linestyle='--', label='Optimal Cluster Count')
plt.legend()
plt.show()

# On retrouve nos 4 clusters
# or do we ?

optimal_cluster_count = 5

# Extract cluster labels
cluster_labels_dendro_5 = fcluster(linkage_matrix, t=optimal_cluster_count, criterion='maxclust')

# Compare dendro clusters to kmeans clusters
labels_kmeans_sample = assign_labels_using_kmeans(alea=42, df=dendro_df, features=features_RFMS, best_k=5, silhouette=True)
ari = adjusted_rand_score(labels_kmeans_sample, cluster_labels_dendro_5)

print(f'ARI = {ari}', '\n')

# Silhouette kmeans = 0.32
# ARI = 0.6, les 2 partitionnements semblent assez différents


### Qualité des clusters


In [ ]:
# Enfin, évaluons la qualité des clusters obtenus

def evaluate_dendro_clusters(linkage='ward'):
    silhouette_scores = []

    for n_clusters in range(2, 10):
        # Create an AgglomerativeClustering model with the chosen linkage method and the current number of clusters
        model = AgglomerativeClustering(n_clusters=n_clusters, linkage=linkage)

        # Fit the model to your data
        cluster_labels = model.fit_predict(dendro_df)

        # Calculate the silhouette score for the current clustering
        silhouette_avg = silhouette_score(dendro_df, cluster_labels)
        silhouette_scores.append(silhouette_avg)

    # Plot the silhouette scores
    plt.figure()
    plt.plot(range(2, 10), silhouette_scores, marker='o', linestyle='-', color='b')
    plt.title("Silhouette Score vs. Number of Clusters")
    plt.xlabel("Number of Clusters")
    plt.ylabel("Silhouette Score")
    plt.grid(True)

    # Find the optimal number of clusters (highest silhouette score)
    optimal_clusters = silhouette_scores.index(max(silhouette_scores)) + 2
    plt.axvline(x=optimal_clusters, color='r', linestyle='--', label=f'Optimal Clusters ({optimal_clusters})')
    plt.legend()

    plt.show()


evaluate_dendro_clusters()

# Pour nb clusters > 2, on retrouve encore 5 comme meilleure valeur.
# Le silhouette score du kmeans était meilleur.
# Tant mieux, l'algo le + rapide est aussi le + performant ici.
# En + kmeans peut travailler avec tt le dataset.


In [ ]:
# Testons d'autres méthodes

evaluate_dendro_clusters(linkage='single')

# ??


In [ ]:
evaluate_dendro_clusters(linkage='complete')

# le silhouette score est curieux ici aussi


In [ ]:
evaluate_dendro_clusters(linkage='average')

# c quoi ces silhouettes ??
# Dues à une perte de complexité lors du sampling ??


### Conclusion


In [ ]:
# Le kmeans l'emporte encore.

# - pour la qualité supérieure de son clustering,

# - pour sa capacité à travailler sur le jeu de données entier, rapidement.
# En effet la "limitation" du clustering hiérarchique (sa complexité algorithmique lourde)
# est un problème sur ce dataset important.

# Notons cependant que pour l'étape finale, le modèle choisi ne travaillera plus que sur une tranche
# réduite du dataset, mise à jour régulièrement. Le clustering hiérarchique reste donc une possibilité.
# (Ceci dit, les scores du k-means restent meilleurs)


## 4 DBSCAN


In [ ]:
# "à la main"

# (ajouter des couleurs)

def plot_DBSCAN_3D(df=rfms_scaled[features_RFMS], legend=True, color_set=custom_color_set_5, opacity=1.0):
    # Specify the DBSCAN parameters (eps and min_samples)
    eps = 0.2  # Maximum distance to form a dense cluster
    min_samples = 5  # Minimum number of samples in a neighborhood to be considered a core point

    # Initialize the DBSCAN clustering model
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)

    # Fit the DBSCAN model to the data
    dbscan.fit(df)

    # Extract cluster labels (-1 represents noise points)
    labels = dbscan.labels_
    print(f'DBSCAN propose {len(set(labels))} clusters.')

    # Convert cluster labels to integer and then to string format
    cluster_labels_str = labels.astype(int).astype(str)

    # Create a list of colors based on labels
    if color_set is None:
        cluster_colors = custom_color_set_5[:len(set(labels))]
    else:
        cluster_colors = color_set[:len(set(labels))]

    # Create a 3D scatter plot with custom legend
    fig = px.scatter_3d(df, x='recent_timestamp', y='nb_commandes', z='payment_total', color=cluster_labels_str,
                        color_discrete_sequence=cluster_colors, opacity=opacity)

    fig.update_traces(marker=dict(size=5), showlegend=legend)
    fig.update_layout(title='DBSCAN Clustering in 3D')

    fig.update_layout(
        width=800,  # Specify the width in pixels
        height=600  # Specify the height in pixels
    )
    fig.show()


plot_DBSCAN_3D()

# mm ?

# On passe toujours très vide de 1 cluster à une multide de clusters,
# pour des valeurs de eps assez proches.
# Difficile d'obtenir juste qq clusters qui aient du sens métier.

# MM conclusion qu'en 3 dimensions :
# Nos points sont espacés de manière trop régulière pour le DBSCAN,
# la topologie n'est pas intéressante pour cet algo car
# Il n'y a pas de zone de vide qui sépare naturellement nos données.


In [ ]:
# Avec un genre de gridsearchcv

# Turns out, GridSearchCV has no score really suitable for clustering evaluation
# We need to do a "manual gridsearch"

# Define the range of eps and min_samples values
eps_values = [0.1, 0.2, 0.3]
min_samples_values = [3, 5, 7]

# Initialize lists to store scores and cluster counts
dbscan_scores = []
cluster_counts = []

# Perform manual grid search
for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(rfms_scaled[features_RFMS])
        # Silhouette prend bcp + de tps
        # score = silhouette_score(rfm_log_scaled, labels)
        # Utilisons D-B ici :
        score = davies_bouldin_score(rfms_scaled[features_RFMS], labels)
        dbscan_scores.append(score)
        nb_clusters = len(set(labels))
        cluster_counts.append(nb_clusters)

print('On trouve entre ' + str(min(cluster_counts)) + ' clusters (valeurs élevées pour eps et min_samples)')
print('et ' + str(max(cluster_counts)) + ' clusters (valeurs faibles)')

# Problème : qd le nb de clusters diminue, le score de Davies-Bouldin augmente,
# ce qui est mauvais : cela signifie des clusters moins compacts et/ou moins séparés.
# Il y a donc un compromis à faire entre la qualité du clustering et son utilité métier

# Or on peut vérifier que pour un nb de clusters donné le kmeans obtenait un score bien inférieur (donc meilleur)
# Encore une fois, sur ces données, le kmeans est à la fois + rapide et plus performant

# Reshape the scores and cluster counts for plotting
dbscan_scores = np.array(dbscan_scores).reshape(len(eps_values), len(min_samples_values))
cluster_counts = np.array(cluster_counts).reshape(len(eps_values), len(min_samples_values))

# Plot the results
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(dbscan_scores, interpolation='nearest', cmap=plt.cm.autumn_r)
plt.title("DB Scores")
plt.xticks(np.arange(len(min_samples_values)), min_samples_values)
plt.yticks(np.arange(len(eps_values)), eps_values)
plt.xlabel("min_samples")
plt.ylabel("eps")
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(cluster_counts, interpolation='nearest', cmap=plt.cm.autumn_r)
plt.title("Number of Clusters")
plt.xticks(np.arange(len(min_samples_values)), min_samples_values)
plt.yticks(np.arange(len(eps_values)), eps_values)
plt.xlabel("min_samples")
plt.ylabel("eps")
plt.colorbar()

plt.show()


In [ ]:
# custom gradual cmap,
# higher parameter values

color1 = (1.0, 1.0, 0.3)
color2 = (1.0, 0.1, 0.1)

num_segments = 256

gradient_colors = [color1, color2]
gradual_cmap = LinearSegmentedColormap.from_list("custom_cmap", gradient_colors, N=num_segments)

eps_values = [0.2, 0.3, 0.4]
min_samples_values = [5, 7, 9]

dbscan_scores = []
cluster_counts = []

# Perform manual grid search
for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(rfms_scaled[features_RFMS])
        score = davies_bouldin_score(rfms_scaled[features_RFMS], labels)
        dbscan_scores.append(score)
        nb_clusters = len(set(labels))
        cluster_counts.append(nb_clusters)

print('On trouve entre ' + str(min(cluster_counts)) + ' clusters (valeurs élevées pour eps et min_samples)')
print('et ' + str(max(cluster_counts)) + ' clusters (valeurs faibles)')

dbscan_scores = np.array(dbscan_scores).reshape(len(eps_values), len(min_samples_values))
cluster_counts = np.array(cluster_counts).reshape(len(eps_values), len(min_samples_values))

# Plot the results
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(dbscan_scores, interpolation='nearest', cmap=gradual_cmap)
plt.title("DB Scores")
plt.xticks(np.arange(len(min_samples_values)), min_samples_values)
plt.yticks(np.arange(len(eps_values)), eps_values)
plt.xlabel("min_samples")
plt.ylabel("eps")
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(cluster_counts, interpolation='nearest', cmap=gradual_cmap)
plt.title("Number of Clusters")
plt.xticks(np.arange(len(min_samples_values)), min_samples_values)
plt.yticks(np.arange(len(eps_values)), eps_values)
plt.xlabel("min_samples")
plt.ylabel("eps")
plt.colorbar()

plt.show()

# mm conclusion que précédemment


## Conclusion features RFMS


In [ ]:
# Au final mon idée de couper le dataset en 2 (clients réguliers vs 1 seule commande)
# n'a pas été utile, puisque les outils de clustering repèrent facilement cette disposition des données,
# et en rendent compte dans l'attribution des labels (= l'assignation à un cluster)

# Lorsqu'on prend en compte uniquement ces 4 features,

# le nombre optimal de clusters que nous pouvons faire est de 5 clusters,
# ce qui mieux. Assez pour une segmentation de qualité ?
# Plutôt que de "forcer" un sous-partitionnement sub-optimal, nous pourrions ajouter une cinquième feature,
# toujours la plus pertinente possible pour comprendre nos clients.

# Cependant nous sommes à la limite de nos outils de visualisation, il serait difficile de travailler
# en 5 dim dans un notebook avec mes connaissances actuelles.
# -> autre outil ? Possibilités de contourner le problème ??

# Quelle autre feature aurions-nous choisi ?
# Une feature géographique, pour mettre en lumière des inégalités dans l'implantation sur le territoire ?
# Je trouve les autres features peut-être moins pertinentes que RFMS pour une entreprise comme Olis.
# Après il faudrait vérifier, c juste une hypothèse.

# Sur 4 dimensions comme sur 3, le k-means n'a pas juste été "plus performant" que les 2 autres algos ;
# il les a littéralement laissés dans la poussière ! (/ mis KO)
# car mm en 4 dimensions, notre data est répartie de manière relativement homogène,
# sa topologie est idéale pour le k-means (et vice-versa)


## Annexes / tests


In [ ]:
# gaussian mixture models ?


## t-SNE


In [ ]:
# Le t-SNE ne permet pas d'interpréter les distances ou les tailles des clusters sur le graph obtenu,
# ce n'est donc pas du tout l'outil idéal pour une segmentation de clientèle.
# C juste pour l'essayer !

# Perform t-SNE
tsne = TSNE(n_components=2, perplexity=30, n_iter=300)
tsne_result = tsne.fit_transform(rfms_scaled[features_RFMS])

# Create a DataFrame to hold the t-SNE results
df_tsne = pd.DataFrame(tsne_result, columns=["Dimension 1", "Dimension 2"])

# Visualize the t-SNE results
plt.figure(figsize=(8, 6))
plt.scatter(df_tsne["Dimension 1"], df_tsne["Dimension 2"])
plt.title("t-SNE Visualization")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.show()

# Wow ! De toute beauté !

# Après recherche, j'avais mal compris. Le t-SNE peut aider dans certains cas à détecter des clusters,
# mais ce n'est pas un outil de clusterisation à proprement parler. Il n'assigne pas de labels.
# C'est un outil de réduction dimentionnelle, et pour l'instant on n'en a mm pas besoin,
# on est "seulement" en 3D et on n'ira pas en dimensions très élevées.

# Serait-ce notre groupe des 3% qu'on devine à droite ?
# Peut-être. Aucune idée.


In [ ]:
# Perform t-SNE with 3 components
tsne = TSNE(n_components=3, perplexity=30, n_iter=300)
tsne_result = tsne.fit_transform(rfms_scaled[features_RFMS])

# Create a DataFrame to hold the t-SNE results
df_tsne = pd.DataFrame(tsne_result, columns=["Dimension 1", "Dimension 2", "Dimension 3"])

# Visualize the t-SNE results in 3D
fig = px.scatter_3d(df_tsne, x="Dimension 1", y="Dimension 2", z="Dimension 3")
fig.update_layout(title="t-SNE Visualization (3D)")
fig.show()

# La Terre est vraiment bleue comme une orange !
# La preuve :
